# Regularization

Let's improve our understanding of what impacted **Titanic** passenger's chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [3]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [4]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 1.  Logistic Regression without regularization

❓ Rank the feature by decreasing order of importance according to a simple **non-regularized** Logistic Regression
- Careful, `LogisticRegression` is penalized by default
- Increase `max_iter` to a larger number until model converges

In [15]:
from sklearn.linear_model import LogisticRegression
logmod=LogisticRegression(penalty='none')
logmod.fit(X,y)
logmod.__dict__
features = pd.DataFrame(data = logmod.coef_, columns = logmod.feature_names_in_).T
features.columns = ["coeff"]
features

/home/cherif/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,coeff
pclass,-0.459749
age,-0.027271
sibsp,-0.495062
parch,-0.155874
fare,0.002748
sex_female,2.681634
class_First,0.637844
class_Third,-0.712655
who_child,1.345468
embark_town_Cherbourg,0.421053


❓How do you interpret, in plain english language, the value for the coefficient `sex_female` ?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


In [17]:
from sklearn.linear_model import LogisticRegression
logmod=LogisticRegression(max_iter = 10000, penalty='none')
logmod.fit(X,y)
logmod.__dict__
features = pd.DataFrame(data = logmod.coef_, columns = logmod.feature_names_in_).T
features.columns = ["coeff"]
features

,coeff
pclass,1.202271
age,-0.027604
sibsp,-0.494979
parch,-0.148612
fare,0.002649
sex_female,2.670608
class_First,2.286586
class_Third,-2.387641
who_child,1.335585
embark_town_Cherbourg,-5.363000


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [18]:
top_1_feature = ["embark_town_Queenstown"]

In [19]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/cherif/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/cherif/code/cherifbenham/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_unregularized.py::TestUnregularized::test_top_1 PASSED        [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master


## 2.  Logistic Regression with a L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its feature importance
- By "strongly regularized" we mean "more than sklearn's default applied regularization factor". 
- Default sklearn's values are very useful orders of magnitudes to keep in mind for "scaled features"

In [20]:
from sklearn.linear_model import LogisticRegression
logmod=LogisticRegression(max_iter = 10000, penalty='l2')
logmod.fit(X,y)
logmod.__dict__
features = pd.DataFrame(data = logmod.coef_, columns = logmod.feature_names_in_).T
features.columns = ["coeff"]
features

,coeff
pclass,-0.735882
age,-0.028874
sibsp,-0.452182
parch,-0.121911
fare,0.002660
sex_female,2.530531
class_First,0.319960
class_Third,-0.427440
who_child,1.104372
embark_town_Cherbourg,0.212004


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [21]:
top_2_features = ["sex_female", "who_child"]

#### 🧪 Test your code below

In [22]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/cherif/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/cherif/code/cherifbenham/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_ridge.py::TestRidge::test_top2 PASSED                         [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master


## 2. Logistic Regression with a L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its feature importance

In [28]:
from sklearn.linear_model import LogisticRegression
logmod=LogisticRegression(max_iter = 10000, penalty='l1', solver ='liblinear')
logmod.fit(X,y)
logmod.__dict__
features = pd.DataFrame(data = logmod.coef_, columns = logmod.feature_names_in_).T
features.columns = ["coeff"]
features.sort_values(by="coeff")

,coeff
class_Third,-0.957156
sibsp,-0.454678
pclass,-0.181199
parch,-0.125781
embark_town_Queenstown,-0.107554
age,-0.026505
embark_town_Southampton,0.000000
fare,0.003058
embark_town_Cherbourg,0.264743
class_First,0.846040


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features, you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model ? 
- From now on, we will always regularize our linear models!

In [29]:
zero_impact_features = ["embark_town_Southampton", "fare", "age", "embark_town_Queenstown"]

#### 🧪 Test your code below

In [30]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/cherif/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/cherif/code/cherifbenham/data-challenges/05-ML/05-Model-Tuning/02-Regularization
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_lasso.py::TestLasso::test_zero_impact PASSED                  [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master


**🏁 Congratulation! Don't forget to commit and push your notebook**